In [6]:
import numpy as np
import scipy.stats as si
import sympy as sp
from sympy.stats import Normal,cdf 
import pandas as pd

E=50 #strike price
rf=0.05
sigma=0.20 #Stock price volatility
T=0.3826 #Time to Maturity

#Black scholes formula를 이용해 Call 옵션을 pricing하는 함수 정의
def BSpricing(St,E,rf,sigma,T):
    d1=(np.log(St/E)+(rf+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2=d1-sigma*np.sqrt(T)
    
    global Ct
    Ct=St*si.norm.cdf(d1,0,1)-E*np.exp(-rf*T)*si.norm.cdf(d2,0,1)
    

# (a)

In [2]:
# t=0일때(현재시점), S0=49인 옵션의 BS price(이론가격)구하기
BSpricing(49,E,rf,sigma,T)

print(round(Ct*100000,3))

239184.247


시장의 call option prices는 \$300,000, BSM으로 계산한 가격은 약 \$239,184로 시중 가격이 고평가 되어있음을 알 수 있다.

# (b)

In [3]:
Dt=300000*np.exp(rf*T)

#(i) when St = $40
payoffC_i= - max(40-E,0)*100000
Strategy1_i=payoffC_i+Dt
print("FI's profit in case of (i): $",round(Strategy1_i,3))

#(ii) when St = $60
payoffC_ii=-max(60-E,0)*100000
Strategy1_ii=payoffC_ii+Dt
print("FI's profit in case of (ii): $",round(Strategy1_ii,3))

FI's profit in case of (i): $ 305794.245
FI's profit in case of (ii): $ -694205.755


# (c)

In [4]:
#(i) when St = $40
shortCi= - max(40-E,0)*100000+300000
longSi=(40-49)*100000

Strategy2_i=shortCi+longSi
print("FI's profit in case of (i): $",Strategy2_i)

#(ii) when St= $60
shortCii=-max(60-E,0)*100000+300000
longSii=(60-49)*100000

Strategy2_ii=shortCii+longSii
print("FI's profit in case of (ii): $",Strategy2_ii)

FI's profit in case of (i): $ -600000
FI's profit in case of (ii): $ 400000


# (d)

In [5]:
N=20
dt=T/N
mu=0.13 # expected return from the stock
S0=49

# 시뮬레이션 값들을 저장하기 위해 각 변수에 대한 빈 행렬을 생성한다.
S=np.zeros(shape=N+1) #Stock
A=np.zeros(shape=N+1) #Asset holding
D=np.zeros(shape=N+1) #Cash Deposit
C=np.zeros(shape=N+1) #Call option
Pi=np.zeros(shape=N+1) #Portfolio value

# 초기값 설정; S0,C0,A0,D0,Pi0 설정
S[0]=S0
BSpricing(49,E,rf,sigma,T)
C[0]=Ct
A[0]= (dt*C[0])/(dt*S0)
D[0]=1

Pi[0]=A[0]*S0+D[0]

# Week0의 첫 줄을 미리 생성한다.
firstLow=[S0,A[0],0,0,0,0]


# OTM과 ITM을 구분할 때 사용할 일주일간의 Spath를 따로 지정
Spath=np.zeros(shape=1000+1)  
bt=dt/1000
Spath[0]=S0


## (i) out-of-the-money

In [13]:
# 값들을 표로 나타낼 OTM프레임 생성 
frameOTM=pd.DataFrame([firstLow],columns=['Stock price', 'Delta', 'Shares purchased', 'Cost of SP', 'cumCost including interest', 'Interest Cost'])

# Delta hedging
for i in range(1,N+1): # 20 weeks
    # stock price path 설정; Observe new asset price
    while True:
        for j in range(1,1000+1):
                z=np.random.standard_normal()
                Spath[j]=Spath[j-1]*np.exp((mu-0.5*sigma**2)*bt+sigma*np.sqrt(bt)*z)
        if Spath[j]<E:#Out-of-the-money
            S[i]=Spath[j]
            break
    
    # Call option price path 설정
    TM=T-i*dt
    BSpricing(S[i],E,rf,sigma,TM)
    C[i]=Ct
    
    Pi[i]=A[i-1]*S[i]+(1+rf*dt)*D[i-1] #Compute new portfolio Pi 
    
    A[i]=sp.limit((C[i]-C[i-1])/(S[i]-S[i-1]),S[i]-S[i-1],0) #Compute new asset holding
    D[i]=(1+rf*dt)*D[i-1]+(A[i-1]-A[i])*S[i] #Compute new Cash Deposit holding

    Pi[i]=A[i]*S[i]+D[i]
    
    sharePurchased=A[i-1]-A[i]
    interestCost=(1+rf*dt)*D[i-1]
    
    newLow=[round(S[i],3),round(A[i],3),round(sharePurchased,3),round(S[i]*sharePurchased,3),round(D[i],3),round(interestCost,3)]
    newFrame=pd.DataFrame([newLow],columns=['Stock price', 'Delta', 'Shares purchased', 'Cost of SP', 'cumCost including interest', 'Interest Cost'])

    frameOTM=frameOTM.append(newFrame,ignore_index=True)
    
    
frameOTM

C:\Users\hoyah\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in sqrt
  
C:\Users\hoyah\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in sqrt
  from ipykernel import kernelapp as app
C:\Users\hoyah\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\hoyah\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\hoyah\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


,Stock price,Delta,Shares purchased,Cost of SP,cumCost including interest,Interest Cost
0,49.000,0.048813,0.000,0.000,0.000,0.000
1,48.248,0.603000,-0.554,-26.717,-25.716,1.001
2,48.157,1.353000,-0.750,-36.123,-61.863,-25.740
3,49.168,0.403000,0.949,46.671,-15.252,-61.922
4,48.556,0.640000,-0.236,-11.478,-26.744,-15.266
5,47.170,0.476000,0.163,7.709,-19.061,-26.770
6,49.430,0.404000,0.072,3.555,-15.524,-19.079
7,46.886,0.459000,-0.054,-2.541,-18.080,-15.539
8,48.702,0.338000,0.120,5.863,-12.235,-18.097
9,48.521,0.989000,-0.650,-31.551,-43.798,-12.246


## (ii) in-the-money

In [14]:
# 값들을 표로 나타낼 ITM프레임 생성
frameITM=pd.DataFrame([firstLow],columns=['Stock price', 'Delta', 'Shares purchased', 'Cost of SP', 'cumCost including interest', 'Interest Cost'])

# Delta hedging
for i in range(1,N+1): # 20 weeks
    # stock price path 설정; Observe new asset price
    while True:
        for j in range(1,1000+1):
                z=np.random.standard_normal()
                Spath[j]=Spath[j-1]*np.exp((mu-0.5*sigma**2)*bt+sigma*np.sqrt(bt)*z)
        if Spath[j]>E:#in-the-money
            S[i]=Spath[j]
            break
    
    # Call option price path 설정
    TM=T-i*dt
    BSpricing(S[i],E,rf,sigma,TM)
    C[i]=Ct
    
    Pi[i]=A[i-1]*S[i]+(1+rf*dt)*D[i-1] #Compute new portfolio Pi 
    
    A[i]=sp.limit((C[i]-C[i-1])/(S[i]-S[i-1]),S[i]-S[i-1],0)
    D[i]=(1+rf*dt)*D[i-1]+(A[i-1]-A[i])*S[i] #Compute new Cash Deposit holding

    Pi[i]=A[i]*S[i]+D[i]
    
    sharePurchased=A[i-1]-A[i]
    interestCost=(1+rf*dt)*D[i-1]

    newLow=[round(S[i],3),round(A[i],3),round(sharePurchased,3),round(S[i]*sharePurchased,3),round(D[i],3),round(interestCost,3)]
    newFrame=pd.DataFrame([newLow],columns=['Stock price', 'Delta', 'Shares purchased', 'Cost of SP', 'cumCost including interest', 'Interest Cost'])

    frameITM=frameITM.append(newFrame,ignore_index=True)
    
    
frameITM

C:\Users\hoyah\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in sqrt
  
C:\Users\hoyah\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in sqrt
  from ipykernel import kernelapp as app
C:\Users\hoyah\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\hoyah\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\hoyah\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1738: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= self.b) & cond0


,Stock price,Delta,Shares purchased,Cost of SP,cumCost including interest,Interest Cost
0,49.000,0.048813,0.000,0.000,0.000,0.000
1,51.749,0.576000,-0.527,-27.280,-26.279,1.001
2,50.891,0.769000,-0.193,-9.831,-36.135,-26.304
3,50.457,0.834000,-0.065,-3.274,-39.444,-36.170
4,50.695,0.228000,0.606,30.727,-8.755,-39.482
5,51.655,0.558000,-0.330,-17.041,-25.804,-8.763
6,50.315,0.716000,-0.158,-7.960,-33.788,-25.828
7,50.805,0.410000,0.306,15.567,-18.254,-33.820
8,50.725,1.903000,-1.494,-75.766,-94.037,-18.271
9,50.669,2.533000,-0.630,-31.903,-126.029,-94.127


(i)과 (ii)의 표를 보면, i기의 Cumulative Cost와 (i+1)기의 interest Cost가 같다. 따라서, Delta-hedging을 이용하면 주가가 OTM일때와 ITM일때 상관없이, 이자비용만큼의 비용만 지불하며 포트폴리오를 헷징할 수 있는 장점이 있다.